https://nlp.stanford.edu/IR-book/pdf/06vect.pdf

<script type="text/javascript" async src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/3.2.2/es5/tex-mml-chtml.min.js">
</script>
<script type="text/x-mathjax-config">
 MathJax.Hub.Config({
 tex2jax: {
 inlineMath: [['$', '$'] ],
 displayMath: [ ['$$','$$'], ["\\[","\\]"] ]
 }
 });
</script>
## 6.1：重み付きゾーンスコアリングとその最適化

---

### 🔸 重み付きゾーンスコアリング（Weighted Zone Scoring）

- 文書には複数の **ゾーン**（例：author, title, body）があり、各ゾーンごとにクエリが一致しているかをブール値（$1$ or $0$）で評価する。
- 各ゾーンには重み $g_i ∈ [0, 1]$ を割り当て、文書のスコアは以下の線形結合で与えられる：

  $
  score(q, d) = Σ g_i * s_i
  $

  - $s_i$：ゾーン $i$ にクエリが一致しているか（0 または 1）
  - $g_i$：ゾーン $i$ の重み（全ゾーンの重みは合計で1）

- **例**：  
  クエリ shakespeare に対して：
  - author に一致 → 0.2
  - title に一致 → 0.3
  - body に一致 → 0.5  
  → title と body に一致する文書のスコアは $0.3 + 0.5 = 0.8$

- インデックスを活用した高速なスコア計算アルゴリズム（ZONESCORE）も紹介されている。

---

### 🔸 重みの学習（Learning Weights）

- 各ゾーンの重み $g_i$ は専門家による設定も可能だが、通常は**機械学習によって自動で学習される**。
- 学習には、以下のようなトレーニングデータが使われる：

  $
  Φ_j = (d_j, q_j, r(d_j, q_j))
  $

  - $d_j$：文書、$q_j$：クエリ、$r(d_j, q_j)$：関連性（Relevant / Non-relevant）

- 2ゾーン（タイトルと本文）でのスコア関数の例：

  $
  score(d, q) = g * s_T(d, q) + (1 - g) * s_B(d, q)
  $

- 各 $(d_j, q_j)$ に対してスコアと実際の関連性を比較し、**誤差（2乗誤差）を最小化**することで $g$ を学習する。

---

### 🔸 最適重み g の導出（The Optimal Weight g）

- 4つのパターンでトレーニングデータを分類：
  - $s_T=1, s_B=0$（タイトル一致のみ）
  - $s_T=0, s_B=1$（本文一致のみ）
  - $s_T=0, s_B=0$（一致なし）
  - $s_T=1, s_B=1$（両方一致）

- 各パターンの中で Relevant / Non-relevant の数を数える：
  - $n_{10r}$：タイトル一致のみでRelevant
  - $n_{01n}$：本文一致のみでNon-relevant
  - など

- 最適な重み $g$ は以下で計算される：

  $
  g = (n_{10r} + n_{01n}) / (n_{10r} + n_{10n} + n_{01r} + n_{01n})
  $

- これは、トレーニングデータの誤差を最小にする $g$ を解析的に導出した式。(ゾーンが2つだからできた？)

---

このアプローチは、簡単なブール一致と線形スコアに基づくランキングモデルであり、**機械学習による情報検索の基礎**とも言える構成です。

## 6.2 語の頻度と重み付け（Term Frequency and Weighting）

### 🔸 背景と目的

- これまでは「語が出現するかどうか（0 または 1）」だけでスコアを決めていた。
- 次のステップは、「語が何回現れるか」を使ってスコアに差をつけること。
- 特に自由形式のクエリ（free text query）では、語の出現回数は関連性に大きく関わる。

---

### 🔸 Term Frequency（TF：語の出現回数）

- 各語 $t$ が文書 $d$ に何回登場したかをカウントする。

  $
  tf_{t,d} = 文書 d における語 t の出現回数
  $

- 「Bag of Words（単語の袋）」モデルに基づく。
  - 単語の順番は無視し、出現回数のみを考慮する。

---

### 🔸 Inverse Document Frequency（IDF：逆文書頻度）

- 出現頻度の高い語（例：the, is）は重要度が低くなるように重みを調整する。
- 定義：

  $
  idf_t = log(N / df_t)
  $

  - $N$：全文書数
  - $df_t$：語 $t$ が出現する文書の数（document frequency）

- **珍しい語ほど IDF が高く、よくある語は IDF が低い。**

---

### 🔸 TF-IDF（語の重みの組み合わせ）

- TF と IDF を掛け合わせた代表的なスコア：

  $
  \textrm{tf-idf}_{t,d} = tf_{t,d} × idf_t
  $

- 特徴：
  1. 限られた文書に頻出 → 高スコア（高い識別力）
  2. 多くの文書に登場 → 低スコア（識別力が低い）
  3. 全文書に登場 → 最小スコア（意味がない）

---

### 🔸 文書スコアの計算（Overlap Score）

- クエリ中のすべての語に対し、文書中の TF-IDF を合計する：

  $
  Score(q, d) = Σ_{t∈q} tf-idf_{t,d}
  $

- より厳密なスコアリング手法は、次のセクション（6.3）でベクトル空間モデルとして説明される。



## 6.3 ベクトル空間モデルによるスコアリング（The Vector Space Model for Scoring）

### 🔸 概要

- 文書やクエリを「語の重みベクトル」として表現し、**数値的な類似度スコア**を用いてランキングを行う。
- 中心的な手法は **コサイン類似度（cosine similarity）** に基づく。

---

### 🔸 6.3.1 ドット積（Dot Products）

- 文書 $d$ は語彙に基づいたベクトル $V(d)$ として表現される（通常は TF-IDF を重みとして使用）。
- 文書同士の類似度は以下の式で定義される：

  $
  sim(d1, d2) = \frac{V(d1) · V(d2)}{|V(d1)| × |V(d2)|}
  $

  - 分子：ドット積（同じ語の重みの積の総和）
  - 分母：ユークリッド長（ベクトルの大きさ）の積（＝正規化）

- この手法により、文書の長さに依存せず、**内容的な近さ**だけを比較できる。

---

### 🔸 6.3.2 クエリをベクトルとして扱う（Queries as Vectors）

- クエリも語の重みベクトル $V(q)$ として扱うことで、文書と同様にスコア計算が可能。

  $
  score(q, d) = \frac{V(q) · V(d)}{|V(q)| × |V(d)|}
  $

- クエリと文書の類似度スコアが高いほど、文書のランキングが上がる。

- 例：  
  クエリ jealous gossip → ベクトル $(0, 0.707, 0.707)$  (affection, jealous, gossip)に相当、単位ベクトル化されている  
  このベクトルと各文書ベクトルの内積（コサイン類似度）を取ることでランキングを得る。

---

### 🔸 6.3.3 ベクトルスコアの計算（Computing Vector Scores）

- term-at-a-time（語ごとにスコアを加算）で各文書のスコアを累積計算。
- 各文書に対してスコアを格納する配列 $Scores[]$ を使い、クエリ内の全語について加算していく。

- アルゴリズムの概要：

  1. クエリ内の各語について重みを計算（例：IDF）
  2. その語が含まれる文書ごとに $Scores[doc] $+=$ \textrm{tf-idf} × idf$
  3. 最終的に正規化（文書ベクトルの長さで割る）
  4. スコア上位の文書を $Top K$ として返す

- **累積配列（accumulator）** により効率的にスコアを管理可能。

---

### 🔸 ベクトル空間モデルの利点

- 文書とクエリを同一の数値ベクトル空間で扱えるため、定量的かつ高速な比較が可能。
- 検索クエリだけでなく、「この文書に似た他の文書を探す」といった**類似文書検索**にも応用できる。
- TF-IDFやコサイン類似度に基づいた**実用的かつ直感的なスコアリング方式**。

